In [ ]:
#Don't forget to shut down kernel by running the cell of the bottom.
#Don't modify this cell.
#Loading the file
import pandas as pd

FILE_NAME = "test.szf"

df = pd.read_csv(FILE_NAME, sep = "\t")
print(df)

In [ ]:
#Enter the starting row and column for the quantified value.
START_ROW = 4
START_COL = 2
COMPOUND_ID_NAME = "InChi"

#PCA
from sklearn.decomposition import PCA 
pca = PCA(n_components = 2)
df_values = df.iloc[START_ROW:, START_COL:].astype(float)
df_values = df_values.apply(lambda x: (x - x.mean()) / x.std(), axis=1) #Auto Scaling
df_values["Name"] = df["Name"][START_ROW:, ]
df_values[COMPOUND_ID_NAME] = df[COMPOUND_ID_NAME][START_ROW:, ]
df_values = df_values.dropna() #Missing value processing
drop_col = ["Name", COMPOUND_ID_NAME]
x = pca.fit_transform(df_values.drop(drop_col, axis=1).T)
embed = pd.DataFrame(x)

pc1_variance = pca.explained_variance_ratio_[0] * 100
pc2_variance = pca.explained_variance_ratio_[1] * 100

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(6, 12))

ax1 = fig.add_subplot(2, 1, 1)
ax1.scatter(embed.iloc[:, 0], embed.iloc[:, 1])
for x, y, name in zip(embed.iloc[:, 0], embed.iloc[:, 1], df_values.drop(drop_col, axis=1).columns[0:]):
    ax1.annotate(name, xy = (x, y))
ax1.grid()
ax1.set_title("Score Plot")
ax1.set_xlabel("PC1({:.1f}%)".format(pc1_variance))
ax1.set_ylabel("PC2({:.1f}%)".format(pc2_variance))

import numpy as np
loading = pca.components_*np.c_[np.sqrt(pca.explained_variance_)]
loading_plot = pd.DataFrame({"PC1" : loading[0], "PC2" : loading[1], "Name" : df_values["Name"], COMPOUND_ID_NAME : df_values[COMPOUND_ID_NAME]})

ax2 = fig.add_subplot(2, 1, 2)
ax2.scatter(loading_plot["PC1"], loading_plot["PC2"])
for x, y, name in zip(loading_plot["PC1"], loading_plot["PC2"], loading_plot["Name"]):
    ax2.annotate(name, xy = (x, y))
ax2.grid()
ax2.set_title("Loading Plot")
ax2.set_xlabel("PC1({:.1f}%)".format(pc1_variance))
ax2.set_ylabel("PC2({:.1f}%)".format(pc2_variance))

fig.tight_layout()
plt.show()

In [ ]:
for i in ["PC1", "PC2"]:
    loading_plot = loading_plot.sort_values(i, ascending=False)
    plt.figure(figsize=(20, 6))
    plt.bar(range(len(loading_plot)), loading_plot[i], tick_label = loading_plot["Name"])
    plt.xticks(rotation=90)
    plt.title("Loading Plot(" + i + ")")
    plt.xlabel("Compound Name")
    plt.ylabel(i)
    plt.show()

In [ ]:
TOP = 10

loading_plot = loading_plot.reindex(loading_plot.PC1.abs().sort_values(ascending=False).index)
pc1 = loading_plot[COMPOUND_ID_NAME][:TOP]

loading_plot = loading_plot.reindex(loading_plot.PC2.abs().sort_values(ascending=False).index)
pc2 = loading_plot[COMPOUND_ID_NAME][:TOP]

output = pd.concat([pc1,pc2]).drop_duplicates().reset_index(drop=True)
#Creating an output file
output.to_csv("output.txt", sep = "\t", index = False, header=False)

In [ ]:
#shutting down the kernel
import os
os.rename("../Terminate/OFF", "../Terminate/ON")